In [1]:
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import mpl_toolkits.axisartist.floating_axes as floating_axes
import mpl_toolkits.axisartist.angle_helper as angle_helper
from mpl_toolkits.axisartist.grid_finder import MaxNLocator
from matplotlib.transforms import Affine2D
from scipy.integrate import quad as quad
%matplotlib notebook

from astropy import constants as const
from astropy.table import Table
from astropy.io import fits
import sys
sys.path.insert(0, "/scratch/ierez/IGMCosmo/VoidFinder/python/")
import voidfinder
#from voidfinder.voidfinder_functions import xyz_to_radecz #THERE IS NO SUCH FUNCTION

In [2]:
#copy pasted from github

RtoD = 180./np.pi
DtoR = np.pi/180.
dec_offset = -90
def to_array(table):
    '''Convert table to numpy array'''

    array = np.array([table['x'], table['y'], table['z']])
    array = array.T

    return array

def xyz_to_radecz(sphere_table):
    '''
    Calculate the ra, dec coordinates for the centers of each sphere
    '''

    r = np.linalg.norm(to_array(sphere_table), axis=1)
    sphere_table['r'] = r.T
    sphere_table['ra'] = np.arctan(sphere_table['y']/sphere_table['x'])*RtoD
    sphere_table['dec'] = np.arcsin(sphere_table['z']/sphere_table['r'])*RtoD

    # Adjust ra value as necessary
    boolean = np.logical_and(sphere_table['y'] != 0, sphere_table['x'] < 0)
    sphere_table['ra'][boolean] += 180.

    return sphere_table

# Data files

### Galaxies

In [3]:
data_directory = '/scratch/ierez/IGMCosmo/VoidFinder/data/DR16S82_H/delta_fields/'

# Original SDSS DR7 (KIAS-VAGC)
deltafields_filename = data_directory + 'deltafields_RAadded90.fits'

quasars_filename=data_directory + 'quasars.fits'

#downDR7_filename = data_directory + 'kias1033_5_LOWZ.txt'

quasars_filename='/scratch/sbenzvi_lab/boss/dr16/delta_fields/quasars.fits'

### Voids

In [4]:
void_directory = '/scratch/ierez/IGMCosmo/VoidFinder/outputs/delta_runs/before_names_20/'

# Original DR7 voids
void_filename = void_directory + 'deltafields_RAadded90._comoving_holes_noMagCut.txt'

# Downsampled SDSS DR7 voids
#downDR7void_filename = void_directory + 'kias1033_5_LOWZ_comoving_holes.txt'

# Import data

### Galaxies

In [5]:
# Original SDSS DR7
#dr7_galaxies = Table.read(dr7_filename, 
#                          format='ascii.commented_header', 
#                          include_names=['ra', 'dec', 'redshift', 'delta'])

deltaf=fits.open(deltafields_filename)  

deltaf=Table(deltaf[1].data)

deltaf['z'].name='redshift'

quasars=fits.open(quasars_filename) 
quasars=Table(quasars[1].data)
quasars
# Downsampled SDSS DR7
#dr7LOWZ_galaxies = Table.read(downDR7_filename, 
#                              format='ascii.commented_header', 
#                              include_names=['ra', 'dec', 'redshift', 'Rgal', 'rabsmag'])

FileNotFoundError: [Errno 2] No such file or directory: '/scratch/sbenzvi_lab/boss/dr16/delta_fields/quasars.fits'

##### Reduce original DR7 to magnitude-limited survey

In [ ]:
#mag_cut = dr7_galaxies['rabsmag'] <= -20
#z_cut = dr7_galaxies['redshift'] < 0.1026

#vollim_dr7_galaxies = dr7_galaxies[np.logical_and(mag_cut, z_cut)]
#vollim_dr7_galaxies = dr7_galaxies

### Voids

In [ ]:
# Original DR7 voids
voids = Table.read(void_filename, format='ascii.commented_header')

# Downsampled SDSS DR7 voids
#dr7LOWZ_voids = Table.read(downDR7void_filename, format='ascii.commented_header')

##### Convert x,y,z to ra,dec,z for voids

In [ ]:
voids = xyz_to_radecz(voids)
#dr7LOWZ_voids = xyz_to_radecz(dr7LOWZ_voids)

# Convert redshift to comoving distance

Going to plot $comoving distance$ in Mpch-1
At these high redshifts, cz is not meaningful. So plot for comoving distance instead.

In [ ]:
c = const.c.to('km/s').value
#z to comoving
#voidfinder.distance.z_to_comoving_dist

Omega_M = 0.3147
c = const.c.to('km/s').value
def Distance(z,omega_m = Omega_M,h = 1):
    dist = np.ones(len(z))
    H0 = 100*h
    for i,redshift in enumerate(z):
        a_start = 1/(1+redshift)
        I = quad(f,a_start,1,args=omega_m)
        dist[i] = I[0]*(c/H0)
    return dist

def f(a,omega_m = Omega_M):
     return 1/(np.sqrt(a*omega_m*(1+((1-omega_m)*a**3/omega_m))))

quasars['r']= Distance(quasars['z'],Omega_M,1)
#Continue with voids['r']

deltaf['r']= Distance(deltaf['redshift'],Omega_M,1)
#dr7LOWZ_galaxies['cz'] = c*dr7LOWZ_galaxies['redshift']

## Check inside the tables

In [ ]:
deltaf[0:5]

In [ ]:
voids[0:5]

In [ ]:
quasars[0:5]

In [ ]:
max(voids['ra'])

In [ ]:
max(quasars['ra'])

In [ ]:
max(deltaf['ra'])

In [ ]:
voids['ra']=voids['ra']-90
deltaf['ra']=deltaf['ra']-90

# Function to set up axes

Following function from declination_slice.ipynb

In [ ]:
def setup_axes3(fig, rect, ra0, ra1, cz0, cz1):
    
    # rotate a bit for better orientation
    #tr_rotate = Affine2D().translate(-90, 0) #already fixed ra
    # scale degree to radians
    tr_scale = Affine2D().scale(np.pi/180, 1)

    #tr = tr_rotate + tr_scale + mpl.projections.PolarAxes.PolarTransform()
    tr = tr_scale + mpl.projections.PolarAxes.PolarTransform()
    
    grid_locator1 = angle_helper.LocatorHMS(4)
    tick_formatter1 = angle_helper.FormatterHMS()
    
    grid_locator2 = MaxNLocator(3)
    
    grid_helper = floating_axes.GridHelperCurveLinear(tr, extremes=(ra0, ra1, cz0, cz1),
                                                      grid_locator1=grid_locator1,
                                                      grid_locator2=grid_locator2,
                                                      tick_formatter1=tick_formatter1,
                                                      tick_formatter2=None)

    ax1 = floating_axes.FloatingSubplot(fig, rect, grid_helper=grid_helper)
    fig.add_subplot(ax1)
    
    # adjust axis
    ax1.axis['left'].set_axis_direction('bottom')
    
    ax1.axis['right'].set_axis_direction('top')
    
    ax1.axis['bottom'].set_visible(False)
    
    ax1.axis['top'].set_axis_direction("bottom")
    ax1.axis['top'].toggle(ticklabels=True, label=True)
    ax1.axis['top'].major_ticklabels.set_axis_direction("top")
    ax1.axis['top'].label.set_axis_direction("top")
    
    #ax1.axis['left'].major_ticklabels.set_axis_direction('right')
    ax1.axis["left"].label.set_text("Comoving Distance [Mpc/h]")
    
    aux_ax = ax1.get_aux_axes(tr)

    aux_ax.patch = ax1.patch  
    ax1.patch.zorder = 0.9  

    return ax1, aux_ax

# Wedge plot

In [ ]:
# 'd' is declination and 's' is the thickness of the declination slice
# Vary 'd' and 's' to get different slices
d,s = -1.25, 2.5 #s=2.5

# slice center, can use this for analysis since the slice thickness is very small
slice_ = d + 0.5*s

### Remove galaxies outside declination range

In [ ]:
deltaf_dec_cut = np.logical_and(deltaf['dec'] > d, 
                                deltaf['dec'] <= d + s)

#dr7LOWZ_dec_cut = np.logical_and(dr7LOWZ_galaxies['dec'] > d, 
#                                 dr7LOWZ_galaxies['dec'] <= d + s)
#
#vollim_dr7_dec_cut = np.logical_and(vollim_dr7_galaxies['dec'] > d, 
#                                    vollim_dr7_galaxies['dec'] <= d + s)
#
deltaf_dec_slice = deltaf[deltaf_dec_cut]
#dr7LOWZ_dec_slice = dr7LOWZ_galaxies[dr7LOWZ_dec_cut]
#vollim_dr7_dec_slice = vollim_dr7_galaxies[vollim_dr7_dec_cut]

quasars_dec_cut = np.logical_and(quasars['dec'] > d, 
                             quasars['dec'] <= d + s)

quasars_slice = quasars[quasars_dec_cut]

### Remove galaxies outside distance range

In [ ]:
cz_min, cz_max = min(deltaf['r']), max(quasars['r']) #32700

deltaf_dist_boolean = np.logical_and(deltaf_dec_slice['r'] > cz_min, 
                                  deltaf_dec_slice['r'] <= cz_max)

#dr7LOWZ_dist_boolean = np.logical_and(dr7LOWZ_dec_slice['cz'] > cz_min, 
#                                      dr7LOWZ_dec_slice['cz'] <= cz_max)
#
#vollim_dr7_dist_boolean = np.logical_and(vollim_dr7_dec_slice['cz'] > cz_min, 
#                                         vollim_dr7_dec_slice['cz'] <= cz_max)
#
deltaf_sample = deltaf_dec_slice[deltaf_dist_boolean]
#dr7LOWZ_sample = dr7LOWZ_dec_slice[dr7LOWZ_dist_boolean]
#vollim_dr7_sample = vollim_dr7_dec_slice[vollim_dr7_dist_boolean]


### Voids that intersect the slice

In [ ]:
deltaf_dec_intersect_cut = np.sin(np.abs(slice_ - voids['dec'])*np.pi/180)*voids['r'] <= voids['radius']
#dr7LOWZ_dec_intersect_cut = np.sin(np.abs(slice_ - dr7LOWZ_voids['dec'])*np.pi/180)*dr7LOWZ_voids['r'] <= dr7LOWZ_voids['radius']

void_sample = voids[deltaf_dec_intersect_cut]
#dr7LOWZ_void_sample = dr7LOWZ_voids[dr7LOWZ_dec_intersect_cut]

# Radius of void at intersection
void_sample['radius_intersect'] = np.sqrt(void_sample['radius']**2 
                                              - (np.sin(np.abs(slice_ - void_sample['dec'])*np.pi/180)*void_sample['r'])**2)

#dr7LOWZ_void_sample['radius_intersect'] = np.sqrt(dr7LOWZ_void_sample['radius']**2 
#                                                  - (np.sin(np.abs(slice_ - dr7LOWZ_void_sample['dec'])*np.pi/180)*dr7LOWZ_void_sample['r'])**2)

In [ ]:
min(quasars['ra'])

In [ ]:
fig = plt.figure(figsize=(15,10))

# Range of ra to use in plot
ra_min, ra_max = -45, 45

ax, aux_ax = setup_axes3(fig, 111, ra_min, ra_max, cz_min, cz_max)


# Original DR7 galaxies
aux_ax.plot(deltaf_sample['ra'], deltaf_sample['r'], 
            '.', c='grey', alpha=0.5, ms=3, zorder=2, label='Delta fields')

# Volume-limited DR7 galaxies
#aux_ax.plot(vollim_dr7_sample['ra'], vollim_dr7_sample['cz'], 
#            '.', c='g', zorder=2, label='Volume-limited DR7')

# Downsampled DR7 galaxies
#aux_ax.plot(dr7LOWZ_sample['ra'], dr7LOWZ_sample['cz'], 
#            '.', c='m', zorder=3, label='DR7 LOWZ')

# Original voids
aux_ax.scatter(void_sample['ra'], void_sample['r'], 
               s=void_sample['radius_intersect']*100, 
               edgecolors='none', facecolors='b', alpha=0.1, 
               zorder=1, label='Voids')

aux_ax.plot(quasars['ra'], 
            quasars['r'], 
            '.', c=(1,135/255,123/255), ms=3, zorder=3, label='Quasars')
# Downsampled voids
#aux_ax.scatter(dr7LOWZ_void_sample['ra'], dr7LOWZ_void_sample['r']*100, 
#               s=dr7LOWZ_void_sample['radius_intersect']*100, 
#               edgecolors='none', facecolors='tab:orange', alpha=0.1, 
#               zorder=5, label='DR7 LOWZ voids')

ax.legend(bbox_to_anchor=(1.1, 1.05),loc='lower center')

mpl.rc('font', size=10)
plt.title('All holes void map with comoving distance metric and DEC$\in$['+str(d)+','+str(d+s)+']')
plt.tight_layout();

## Thinner slice

In [ ]:
# 'd' is declination and 's' is the thickness of the declination slice
# Vary 'd' and 's' to get different slices
d,s = 0, 0.05 #s=2.5

# slice center, can use this for analysis since the slice thickness is very small
slice_ = d + 0.5*s

In [ ]:
deltaf_dec_cut = np.logical_and(deltaf_galaxies['dec'] > d, 
                             deltaf_galaxies['dec'] <= d + s)

#dr7LOWZ_dec_cut = np.logical_and(dr7LOWZ_galaxies['dec'] > d, 
#                                 dr7LOWZ_galaxies['dec'] <= d + s)
#
#vollim_dr7_dec_cut = np.logical_and(vollim_dr7_galaxies['dec'] > d, 
#                                    vollim_dr7_galaxies['dec'] <= d + s)
#
deltaf_dec_slice = deltaf[deltaf_dec_cut]
#dr7LOWZ_dec_slice = dr7LOWZ_galaxies[dr7LOWZ_dec_cut]
#vollim_dr7_dec_slice = vollim_dr7_galaxies[vollim_dr7_dec_cut]
#
quasars_dec_cut = np.logical_and(quasars['dec'] > d, 
                             quasars['dec'] <= d + s)

quasars_slice = quasars[quasars_dec_cut]

In [ ]:
cz_min, cz_max = min(deltaf['r']), max(quasars['r']) #32700

deltaf_dist_boolean = np.logical_and(deltaf_dec_slice['r'] > cz_min, 
                                     deltaf_dec_slice['r'] <= cz_max)

#dr7LOWZ_dist_boolean = np.logical_and(dr7LOWZ_dec_slice['cz'] > cz_min, 
#                                      dr7LOWZ_dec_slice['cz'] <= cz_max)
#
#vollim_dr7_dist_boolean = np.logical_and(vollim_dr7_dec_slice['cz'] > cz_min, 
#                                         vollim_dr7_dec_slice['cz'] <= cz_max)
#
deltaf_sample = deltaf_dec_slice[deltaf_dist_boolean]
#dr7LOWZ_sample = dr7LOWZ_dec_slice[dr7LOWZ_dist_boolean]
#vollim_dr7_sample = vollim_dr7_dec_slice[vollim_dr7_dist_boolean]


In [ ]:
deltaf_dec_intersect_cut = np.sin(np.abs(slice_ - voids['dec'])*np.pi/180)*voids['r'] <= voids['radius']
#dr7LOWZ_dec_intersect_cut = np.sin(np.abs(slice_ - dr7LOWZ_voids['dec'])*np.pi/180)*dr7LOWZ_voids['r'] <= dr7LOWZ_voids['radius']

void_sample = voids[deltaf_dec_intersect_cut]
#dr7LOWZ_void_sample = dr7LOWZ_voids[dr7LOWZ_dec_intersect_cut]

# Radius of void at intersection
void_sample['radius_intersect'] = np.sqrt(void_sample['radius']**2 
                                              - (np.sin(np.abs(slice_ - void_sample['dec'])*np.pi/180)*void_sample['r'])**2)

#dr7LOWZ_void_sample['radius_intersect'] = np.sqrt(dr7LOWZ_void_sample['radius']**2 
#                                                  - (np.sin(np.abs(slice_ - dr7LOWZ_void_sample['dec'])*np.pi/180)*dr7LOWZ_void_sample['r'])**2)

In [ ]:
dpi=500
mpl.rcParams['figure.dpi']= dpi
fig = plt.figure(figsize=(15,10))

# Range of ra to use in plot
ra_min, ra_max = -45, 45

ax, aux_ax = setup_axes3(fig, 111, ra_min, ra_max, cz_min, cz_max)


# Original DR7 galaxies
aux_ax.plot(deltaf_sample['ra'], deltaf_sample['r'], 
            '.', c='grey', alpha=0.5, ms=3, zorder=2, label='Delta fields')

# Volume-limited DR7 galaxies
#aux_ax.plot(vollim_dr7_sample['ra'], vollim_dr7_sample['cz'], 
#            '.', c='g', zorder=2, label='Volume-limited DR7')

# Downsampled DR7 galaxies
#aux_ax.plot(dr7LOWZ_sample['ra'], dr7LOWZ_sample['cz'], 
#            '.', c='m', zorder=3, label='DR7 LOWZ')

# Original voids
aux_ax.scatter(void_sample['ra'], void_sample['r'], 
               s=void_sample['radius_intersect']*100, 
               edgecolors='none', facecolors='b', alpha=0.1, 
               zorder=1, label='Voids')

aux_ax.plot(quasars['ra'], 
            quasars['r'], 
            '.', c=(1,135/255,123/255), ms=3, zorder=3, label='Quasars')
# Downsampled voids
#aux_ax.scatter(dr7LOWZ_void_sample['ra'], dr7LOWZ_void_sample['r']*100, 
#               s=dr7LOWZ_void_sample['radius_intersect']*100, 
#               edgecolors='none', facecolors='tab:orange', alpha=0.1, 
#               zorder=5, label='DR7 LOWZ voids')

ax.legend(bbox_to_anchor=(1.1, 1.05),loc='lower center')

mpl.rc('font', size=10)
plt.title('All holes void map with comoving distance metric and DEC$\in$['+str(d)+','+str(d+s)+']')
plt.tight_layout();

In [ ]:
dpi=500
mpl.rcParams['figure.dpi']= dpi
fig = plt.figure(figsize=(15,8))

# Range of RA to use in plot
ra_min, ra_max = -5, 5

fig = plt.figure(figsize=(15,10))

ax, aux_ax = setup_axes3(fig, 111, ra_min, ra_max, cz_min, cz_max)


# Original DR7 galaxies
aux_ax.plot(deltaf_sample['ra'], deltaf_sample['r'], 
            '.', c='grey', alpha=0.5, ms=3, zorder=2, label='Delta fields')

# Volume-limited DR7 galaxies
#aux_ax.plot(vollim_dr7_sample['ra'], vollim_dr7_sample['cz'], 
#            '.', c='g', zorder=2, label='Volume-limited DR7')

# Downsampled DR7 galaxies
#aux_ax.plot(dr7LOWZ_sample['ra'], dr7LOWZ_sample['cz'], 
#            '.', c='m', zorder=3, label='DR7 LOWZ')

# Original voids
aux_ax.scatter(void_sample['ra'], void_sample['r'], 
               s=void_sample['radius_intersect']*100, 
               edgecolors='none', facecolors='b', alpha=0.1, 
               zorder=1, label='Voids')

aux_ax.plot(quasars['ra'], 
            quasars['r'], 
            '.', c=(1,135/255,123/255), ms=3, zorder=3, label='Quasars')
# Downsampled voids
#aux_ax.scatter(dr7LOWZ_void_sample['ra'], dr7LOWZ_void_sample['r']*100, 
#               s=dr7LOWZ_void_sample['radius_intersect']*100, 
#               edgecolors='none', facecolors='tab:orange', alpha=0.1, 
#               zorder=5, label='DR7 LOWZ voids')

ax.legend(bbox_to_anchor=(1.1, 1.05),loc='lower center')

mpl.rc('font', size=10)
plt.title('All holes void map with comoving distance metric and DEC$\in$['+str(d)+','+str(d+s)+']')
plt.tight_layout();